In [7]:
import pandas as pd
import numpy as np
import itertools as it
from datetime import datetime
import networkx as nx
# Documentacion de la libreria: http://networkx.readthedocs.io/en/networkx-1.11/

from operator import itemgetter
from itertools import groupby

In [8]:
def filterProblems(df, isTraining, date):
    """
        Funcion que devuelve el conjunto de problemas que tienen status AC o PE
        Si isTraining es true, entonces la funcion sacara el training_set, si no, sacara el evaluation_set
        date es la fecha de particion
    """
    
    if isTraining:
        df = df[df['submissionDate'] < date]
        df = df.loc[df['status'].isin(['AC', 'PE'])]
    else:
        df = df[df['submissionDate'] >= date]
    
    

    return df


In [9]:
def compareNodes(f_list, s_list):
    """
        Funcion que devuelve el numero de usuarios que han hecho ambos problemas
    """
    peso = len(np.intersect1d(f_list, s_list))
    
    return peso
    
def createLinks(prob_us_set, nodos):
    """
        Funcion que crea los enlaces del grafo a partir de la informacion contenida en el conjunto que se le
        pasa a la funcion
    """
    resultado = list() 
    
    # hago todas las posibles combinaciones de problemas
    for fst, snd in it.combinations(nodos, 2):
        # obtengo el peso pasando la lista de usuarios que ha hecho cada problema
        peso = compareNodes(prob_us_set[fst], prob_us_set[snd])
        if peso >= 1:
            resultado.append((fst, snd, peso))
            
            
            
    return resultado

In [10]:
def filterWeight(weightUmbral, linksToFilter):
    """
        Funcion que filtra los enlaces de un grafo, para que el peso sea mayor o igual al dado
    """
    
    result = [(x, y, z) for (x, y, z) in linksToFilter if z >= weightUmbral]
    
    return result
    

In [11]:
def create_graph_nx(list_nodes, list_links):
    """
        Funcion que crea un grafo de tipo Graph de la libreria NetworkX
        Construccion del grafo: http://networkx.readthedocs.io/en/networkx-1.11/tutorial/tutorial.html#what-to-use-as-nodes-and-edges
    """
    grafo = nx.Graph() # creo la variable grafo

    # incluyo los nodos del grafo 
    grafo.add_nodes_from(list_nodes)

    # se incluyen las tuplas de enlaces con el peso del enlace
    # es una lista de la forma [(Nodo1, Nodo2, peso), ......]
    grafo.add_weighted_edges_from(list_links)

    return grafo

In [12]:
# MAIN
# ---------

# se guarda en la variable df (DataFrame) toda la base de datos
df = pd.read_csv('bbdd_orderbydate.csv')

# aqui quito los problemas que no existian despues de la fecha umbral
df = df[df['problem_id'] <= 511] 

# construyo el conjunto de entrenamiento
training_set = filterProblems(df, True, "2016-10-21 00:00:00")

print(len(training_set))

# obtengo los nodos del grafo:
nodes = training_set.problem_id.unique()


# creo un diccionario que va a tener a los problemas como keys y los valores seran los
# usuarios que han hecho ese problema
grouped = training_set.groupby('problem_id')['user_id'].apply(list)




10262


In [13]:
print(training_set)

       problem_id  user_id status       submissionDate
0              10        5     AC  2014-02-17 15:27:07
1               2        6     AC  2014-02-17 15:39:17
2               2        9     AC  2014-02-18 00:30:14
3              10        9     AC  2014-02-18 00:34:46
4               4        9     AC  2014-02-18 00:50:28
5               6        9     AC  2014-02-18 00:52:11
6              13        9     AC  2014-02-18 00:53:40
9              15        8     AC  2014-02-19 19:58:03
10              4        8     AC  2014-02-20 14:23:30
11             39       16     AC  2014-02-20 15:44:33
12             39       17     AC  2014-02-20 16:53:34
13             13       12     AC  2014-02-21 11:08:38
15             33       12     AC  2014-02-21 11:58:32
16             39       12     AC  2014-02-21 12:05:49
17             44       12     AC  2014-02-21 12:52:44
18             44       15     AC  2014-02-21 14:51:52
19             44       22     AC  2014-02-21 14:58:55
21        

In [14]:
# OBTENCION DEL EVALUATION_SET
# -------

# ahota saco el evaluation_set
evaluation_set = filterProblems(df, False, "2016-10-21 00:00:00")

print(evaluation_set)

# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user_eval = evaluation_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users_eval = pd.DataFrame({'user_id':grouped_user_eval.index, 'list_problem_id':grouped_user_eval.values})

print(df_users_eval)

       problem_id  user_id status       submissionDate
16939         469      799     RF  2016-10-21 00:08:23
16940         325     3832     AC  2016-10-21 00:08:35
16942         469     3757     AC  2016-10-21 00:41:36
16943         469     3792    RTE  2016-10-21 01:04:08
16944         469      810     AC  2016-10-21 01:48:35
16945         438     3943     AC  2016-10-21 02:23:56
16946         469     3773     AC  2016-10-21 05:00:58
16947         346     3728     AC  2016-10-21 09:06:38
16948         469     3736     CE  2016-10-21 09:38:14
16949         438     3785     TL  2016-10-21 09:43:52
16950         469     3704     AC  2016-10-21 10:04:03
16951         469     2912     AC  2016-10-21 10:20:36
16952         213     3136     AC  2016-10-21 10:28:23
16953         213     3088     AC  2016-10-21 10:29:26
16954         469     3708    RTE  2016-10-21 10:35:15
16956         235     3136    RTE  2016-10-21 10:37:58
16957         183     3880     AC  2016-10-21 11:38:11
16958     

In [15]:

# creo los enlaces a partir de la informacion de los nodos
links = createLinks(grouped, nodes)
# ahora filtro el grafo para que los enlaces solo tengan el peso que quiero
linksFiltered = filterWeight(5, links)

print(len(linksFiltered))

# aqui creo el grafo 
graph = create_graph_nx(nodes, linksFiltered)

10837


In [16]:
def apply_aa(row, graph):
    """
        Funcion que devuelve para cada par de nodos, el sumatorio de 1/log(N(z)), siendo N(z) el grado del nodo z para todo z 
        perteneciente al conjunto de nodos en comun de ese par de nodos
    """
    
    # obtengo un iterador de un solo elemento que tiene en la tercera posicion el valor de AA para el par de nodos
    value = nx.adamic_adar_index(graph, [(row['one'], row['two'])])
    
    value_aa = 0
    for u, v, p in value:
        # itero el iterador, guardando el valor de adar adamic
        value_aa = p
    
    return value_aa

def create_aa_data(graph, nodes):

    # Ahora voy a construir un DataFrame que tenga dos columnas con todas las posibles combinaciones de problemas, y otra 
    # columna con el valor de aa para ese par de problemas
    fst_column = list()
    snd_column = list()
    for fst, snd in it.combinations(nodes, 2):
        fst_column.append(fst)
        snd_column.append(snd)

    d = {'one' : fst_column,
        'two' : snd_column}
    dataFrame_aa = pd.DataFrame(d)


    # Aplico la funcion a cada fila
    dataFrame_aa['aa'] = dataFrame_aa.apply (lambda row: apply_aa(row, graph), axis=1)


    return dataFrame_aa

In [17]:
# Hasta aqui se hace la creacion del grafo
aa_df = create_aa_data(graph, nodes)
print(aa_df)

       one  two         aa
0       10    2  29.481399
1       10    4  29.480613
2       10    6  29.480613
3       10   13  28.835311
4       10   15  28.794947
5       10   39  29.481399
6       10   33  29.273420
7       10   44  29.017307
8       10   19  28.171070
9       10   27  29.480877
10      10   60  24.280302
11      10  100  29.481399
12      10   93  10.584318
13      10   53  28.222100
14      10   51  28.411760
15      10   49  29.257845
16      10   70  28.210133
17      10   62  29.007155
18      10   35  28.373895
19      10   81  25.908110
20      10    8  26.367535
21      10   17  24.092536
22      10   23  28.165266
23      10   25   0.000000
24      10   86  29.481139
25      10   29  27.592215
26      10   31   0.000000
27      10   47   0.000000
28      10   65  28.794680
29      10   73  27.133130
...    ...  ...        ...
14166  510  509  28.831763
14167  510  507  20.839273
14168  511  504   0.000000
14169  511  503  29.041361
14170  511  505  28.417354
1

In [18]:
# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user = training_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users = pd.DataFrame({'user_id':grouped_user.index, 'list_problem_id':grouped_user.values})

print(df_users)

                                        list_problem_id  user_id
0                                     [10, 76, 489, 39]        5
1                                              [2, 487]        6
2                      [39, 44, 76, 105, 109, 114, 117]        7
3     [15, 4, 19, 27, 60, 53, 105, 76, 83, 90, 147, ...        8
4          [2, 10, 4, 6, 13, 253, 57, 62, 143, 83, 109]        9
5                                             [307, 39]       10
6                           [33, 109, 39, 93, 100, 155]       11
7     [13, 33, 39, 44, 100, 51, 49, 70, 81, 53, 130,...       12
8                                         [44, 187, 39]       15
9                                                  [39]       16
10                                             [39, 86]       17
11                            [119, 114, 316, 507, 506]       18
12                                                 [44]       22
13                                            [100, 13]       24
14    [258, 141, 309, 310

In [19]:
def lenProblemsDone(row, set_filter):
    """
        Funcion auxiliar que calcula cuanto problemas ha hecho cada usuario en un conjunto: training o evaluation
    """
    # saco el dataframe que contendra solo una fila con la lista de problemas que ha hecho el usuario
    df_filter = set_filter[set_filter['user_id'] == row['user_id']]
    
    if df_filter.empty:
        # si esta vacio, entonces es que el usuario no ha hecho problemas en ese conjunto
        return 0
    else:
        # sino, devuelvo la longitud de la lista de problemas
        return len(df_filter['list_problem_id'].iloc[0]) 
    
    

In [20]:
# aqui voy a hacer el filtro de usuarios de forma que para hacer las recomendaciones solo tengamos en 
# cuenta aquellos usuarios que han hecho 5 o mas problemas tanto antes de la fecha limite como despues

# primero guardo la lista de usuarios
user_list = df.user_id.unique()

# la meto en un dataframe 
column_user_filter = {'user_id': user_list}
datraframe_user_filter = pd.DataFrame.from_dict(column_user_filter)


# ahora tengo que calcular para cada fila, el numero de problemas que han hecho en el training_set, evaluation_set
datraframe_user_filter['len_training'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users), axis=1)
datraframe_user_filter['len_evaluation'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users_eval), axis=1)
print(datraframe_user_filter)


      user_id  len_training  len_evaluation
0           5             4               1
1           6             2               0
2           9            11               3
3           8            47               0
4          16             1               0
5          17             2               2
6          12            12               0
7          18             5               8
8          15             3               1
9          22             1               0
10          7             7               0
11         27             0               0
12         28             1               0
13         29             1               0
14         32             2               0
15         35            41               1
16         33             9               0
17         39            11               0
18         31             9               0
19         40             1               0
20         44             8               0
21         48             0     

In [21]:
# ahora tengo que hacer el filtro en este dataframe, de forma que solo aparezcan las filas en las que len_training y 
# len_evaluation sea >=5
datraframe_user_filter = datraframe_user_filter[(datraframe_user_filter['len_training'] >= 5) & (datraframe_user_filter['len_evaluation'] >=5)]
print(datraframe_user_filter)

# aqui voy a guardar la lista de usuarios a los que voy a recomendar
user_list_to_recommend = sorted(datraframe_user_filter['user_id'].tolist())
print(user_list_to_recommend)
print(len(user_list_to_recommend))


      user_id  len_training  len_evaluation
7          18             5               8
29         60            18              14
118        25            22               7
261       130            80               6
299       414            49              61
316       443            53               8
317       448            43              21
483       689            33               7
619       912            13              11
633       935           127               9
1196     1711           128              16
1288     1893             7               6
1311     1952            19               9
1339     1955             9               5
1416     2038             5               5
1423     2096            12              13
1446     2051            11               8
1447     2025            12               8
1448     2120             7               8
1451     2041             9               7
1528     2257             6              13
1694     2576             5     

In [22]:
# ahora tengo que filtrar df_users para que solo contenga las filas en las que los usuarios
# pertenecen a la anterior lista

df_users = df_users[df_users['user_id'].isin(user_list_to_recommend)]
print(df_users)

                                        list_problem_id  user_id
11                            [119, 114, 316, 507, 506]       18
14    [258, 141, 309, 310, 390, 187, 510, 438, 13, 2...       25
39    [100, 62, 2, 57, 53, 23, 134, 15, 147, 233, 24...       60
90    [309, 150, 183, 237, 191, 187, 39, 209, 70, 15...      130
241   [213, 2, 109, 114, 10, 436, 438, 437, 404, 239...      414
255   [39, 100, 247, 150, 183, 44, 471, 109, 307, 30...      443
257   [255, 257, 314, 311, 315, 310, 282, 243, 275, ...      448
392   [171, 272, 282, 209, 373, 155, 2, 39, 134, 471...      689
512   [241, 269, 2, 340, 65, 379, 316, 275, 309, 259...      912
523   [65, 241, 239, 105, 231, 269, 275, 331, 257, 1...      935
933   [507, 33, 83, 217, 312, 256, 49, 4, 503, 254, ...     1711
997                      [39, 49, 465, 159, 134, 13, 2]     1893
1021  [325, 441, 237, 183, 438, 510, 226, 4, 327, 21...     1952
1024      [325, 327, 441, 468, 471, 503, 226, 109, 446]     1955
1066  [155, 505, 49, 162,

In [23]:
# con esto creo un dataframe que separa el dataframe anterior
# lista de usuario - problema en el conjunto de entrenamiento
df_new = df_users.groupby('user_id').list_problem_id.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

df_new.columns = ['user_id','problem']

print(df_new)

      user_id  problem
0          18      119
1          18      114
2          18      316
3          18      507
4          18      506
5          25      258
6          25      141
7          25      309
8          25      310
9          25      390
10         25      187
11         25      510
12         25      438
13         25       13
14         25      251
15         25      502
16         25      508
17         25      471
18         25      313
19         25      162
20         25      147
21         25      206
22         25      312
23         25      485
24         25      505
25         25      404
26         25      145
27         60      100
28         60       62
29         60        2
...       ...      ...
1002     3941       44
1003     3941      325
1004     3941      254
1005     3941      327
1006     3941      150
1007     3941      183
1008     3946       44
1009     3946      325
1010     3946      327
1011     3946      150
1012     3946      183
1013     39

In [24]:
def getKrecommendations(row, aa_df, df_users, k):
    """
        Funcion que devuelve la lista de k mejores problemas para el usuario dado teniendo en cuenta que 
        las recomendaciones no son problemas que ya haya realziado el usuario
    """
    #column_result = aa_df[aa_df['one'] == 10].sort_values('aa', ascending=False)
    # user_list = df_users[df_users['user_id'] == 6]
    
   
    
    # primero saco los dos dataframes con problemas que se pueden recomendar
    column_result_one = aa_df[aa_df['one'] == row['problem']]
    column_result_two = aa_df[aa_df['two'] == row['problem']]
    
    
    tmp1 = column_result_two['two'].tolist()
    tmp2 = column_result_two['one'].tolist()
    tmp3 = column_result_two['aa'].tolist()
   
    # creo un nuevo df
    df_tmp = pd.DataFrame({'one':tmp1, 'two':tmp2, 'aa': tmp3})
    
    frames = [column_result_one, df_tmp]
    
    # concateno los resultados
    column_result_tmp = pd.concat(frames)
    
    # ordeno los problemas que se pueden recomendar
    column_result_tmp2 = column_result_tmp.sort_values('aa', ascending=False)
     
    tmp1 = column_result_tmp2['one'].tolist()
    tmp2 = column_result_tmp2['two'].tolist()
    tmp3 = column_result_tmp2['aa'].tolist()
    
    # creo un nuevo df     
    column_result = pd.DataFrame({'one':tmp1, 'two':tmp2, 'aa': tmp3})
    
    # con esto consigo sacar la lista de problemas que ha realizado ese usuario
    user_list = df_users[df_users['user_id'] == row['user_id']]
    user_list = user_list['list_problem_id'].iloc[0] 
    
    # ahora filtro la columna para que los problemas recomendados no los haya hecho ya el usuario
    column_result = column_result[column_result['two'].isin(user_list) == False]
    
    return (column_result['two'].head(k)).tolist()
    

In [25]:
# Aplico la funcion a cada fila 
k = 10
df_new['list_recommendations'] = df_new.apply (lambda row: getKrecommendations(row, aa_df,  df_users, k), axis=1)
print(df_new)


      user_id  problem                               list_recommendations
0          18      119   [109, 128, 100, 258, 254, 256, 143, 171, 2, 255]
1          18      114    [109, 128, 39, 143, 2, 258, 254, 256, 100, 105]
2          18      316   [109, 508, 128, 254, 2, 255, 256, 310, 100, 171]
3          18      507   [355, 275, 259, 228, 27, 109, 508, 128, 39, 310]
4          18      506    [109, 508, 128, 310, 39, 256, 254, 143, 2, 346]
5          25      258    [114, 254, 100, 256, 39, 436, 109, 128, 2, 255]
6          25      141   [109, 114, 128, 171, 2, 143, 100, 255, 256, 254]
7          25      309  [346, 503, 171, 255, 114, 254, 256, 100, 275, 39]
8          25      310    [255, 109, 114, 128, 2, 143, 503, 39, 346, 256]
9          25      390  [114, 256, 100, 254, 39, 503, 441, 436, 468, 445]
10         25      187   [109, 114, 128, 143, 255, 2, 256, 254, 100, 171]
11         25      510    [109, 114, 128, 254, 2, 255, 39, 100, 346, 256]
12         25      438  [109, 114, 128

In [26]:
# creo un nuevo dataframe que agrupa por el primer problema 
df_separation = df_new.groupby(['user_id', 'problem']).list_recommendations.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_2', axis = 1)

df_separation.columns = ['user_id','problem', 'recommendation']

print(df_separation)

       user_id  problem  recommendation
0           18      114           109.0
1           18      114           128.0
2           18      114            39.0
3           18      114           143.0
4           18      114             2.0
5           18      114           258.0
6           18      114           254.0
7           18      114           256.0
8           18      114           100.0
9           18      114           105.0
10          18      119           109.0
11          18      119           128.0
12          18      119           100.0
13          18      119           258.0
14          18      119           254.0
15          18      119           256.0
16          18      119           143.0
17          18      119           171.0
18          18      119             2.0
19          18      119           255.0
20          18      316           109.0
21          18      316           508.0
22          18      316           128.0
23          18      316           254.0


In [27]:
def asignaaavalue(row, aa_df):
    """
        Funcion que devuelve una columna con los valores de aa asociados a cada fila
    """
    
    # print(aa_df.one == row['problem'])
    # print(aa_df.two)
    # print(row['recommendation'])
    # print(aa_df.two == row['recommendation'])
    # print((aa_df.one == row['problem']) & (aa_df.two == row['recommendation']))
    
    new_df = aa_df[(aa_df.one == row['problem']) & (aa_df.two == row['recommendation'])]
    
    if new_df.empty:
        new_df = aa_df[(aa_df.two == row['problem']) & (aa_df.one == row['recommendation'])]
    
    return new_df.iloc[0]['aa']

In [28]:
df_separation['aa'] = df_separation.apply(lambda row: asignaaavalue(row, aa_df), axis=1)
print(df_separation)

       user_id  problem  recommendation         aa
0           18      114           109.0  31.518601
1           18      114           128.0  31.518094
2           18      114            39.0  31.072627
3           18      114           143.0  30.959726
4           18      114             2.0  30.959726
5           18      114           258.0  30.945029
6           18      114           254.0  30.945029
7           18      114           256.0  30.945029
8           18      114           100.0  30.945029
9           18      114           105.0  30.931813
10          18      119           109.0  30.387644
11          18      119           128.0  30.387137
12          18      119           100.0  30.386880
13          18      119           258.0  30.386880
14          18      119           254.0  30.386880
15          18      119           256.0  30.386880
16          18      119           143.0  30.386880
17          18      119           171.0  30.386880
18          18      119        

In [29]:
# ahora lo que quiero es ordenar los problemas por cada usuario en funcion de su aa
# primero ordeno por su valor de user y luego por el de aa, de forma que quedan ordenador por su valor aa
df_separation = df_separation.sort_values(by=['user_id', 'aa'], ascending=False)
print(df_separation)

       user_id  problem  recommendation         aa
10260     3969       39           114.0  31.072627
10261     3969       39           109.0  30.726904
10262     3969       39           128.0  30.726396
10263     3969       39             2.0  30.726139
10264     3969       39           143.0  30.726139
10265     3969       39           256.0  30.711442
10266     3969       39           100.0  30.711442
10267     3969       39           258.0  30.711442
10268     3969       39           254.0  30.711442
10269     3969       39           436.0  30.711182
10270     3969      134           109.0  30.348093
10271     3969      134           114.0  30.347841
10272     3969      134           128.0  30.347586
10273     3969      134           143.0  30.347329
10274     3969      134             2.0  30.347329
10275     3969      134           258.0  30.347329
10276     3969      134           254.0  30.347329
10277     3969      134           171.0  30.347329
10278     3969      134        

In [30]:
# ahora tengo que hacer un nuevo dataframe con usuario, problema, y una lista de recommendation 
# (los tres primeros ya que estan ordenados por aa)

# hago primero la agrupacion por usuario
grouped_r = df_separation.groupby('user_id')

# hago la agregacion en una lista 
df_recommend = grouped_r.aggregate(lambda x:list(x))

print(df_recommend)

                                                   problem  \
user_id                                                      
18       [114, 114, 114, 114, 114, 114, 114, 114, 114, ...   
25       [508, 508, 258, 258, 258, 258, 508, 508, 310, ...   
60       [309, 2, 2, 2, 2, 254, 100, 100, 100, 254, 254...   
130      [309, 109, 109, 39, 2, 143, 143, 2, 503, 100, ...   
414      [508, 109, 114, 508, 114, 143, 2, 508, 114, 11...   
443      [109, 114, 508, 109, 114, 2, 2, 114, 100, 254,...   
448      [310, 310, 310, 310, 310, 508, 508, 508, 508, ...   
689      [109, 114, 109, 114, 2, 2, 114, 114, 114, 114,...   
912      [309, 2, 2, 2, 2, 2, 2, 309, 275, 2, 2, 2, 2, ...   
935      [114, 109, 128, 2, 143, 100, 254, 256, 258, 43...   
1711     [508, 346, 503, 171, 255, 310, 114, 254, 256, ...   
1893     [39, 2, 2, 2, 2, 2, 39, 39, 39, 39, 39, 39, 39...   
1952     [346, 143, 143, 143, 143, 143, 327, 143, 441, ...   
1955     [109, 109, 109, 109, 503, 109, 109, 109, 503, ...   
2025    

In [31]:
# para sacar el dataframe final con user - krecomendaciones
del df_recommend['problem']
del df_recommend['aa']

In [32]:
def delRepetitions(row):
    """
        Funcion auxiliar para evitar que salgan repeticiones en las recomendaciones. Saco la lista de posibles 
        recomendaciones con valores unicos
    """
    conjunto_vacio = set()
    
    # esto sirve para que se haga mas rapido la comprobacion de si el elemento esta en la lista o no
    function_add = conjunto_vacio.add
    
    # hago la lista intensional, para mantener el orden dado en la lista original
    return [x for x in row['recommendation'] if not (x in conjunto_vacio or function_add(x))]

In [33]:
# ahora voy a aplicar una funcion a la lista de posibles recomendaciones, para quitar los repetidos
df_recommend['recommendation'] = df_recommend.apply(lambda row: delRepetitions(row), axis=1)
print(df_recommend)

                                            recommendation
user_id                                                   
18       [109.0, 128.0, 39.0, 143.0, 2.0, 258.0, 254.0,...
25       [346.0, 503.0, 114.0, 254.0, 100.0, 256.0, 171...
60       [508.0, 109.0, 114.0, 128.0, 143.0, 256.0, 258...
130      [508.0, 114.0, 128.0, 256.0, 254.0, 258.0, 255...
414      [309.0, 128.0, 346.0, 39.0, 503.0, 256.0, 100....
443      [128.0, 346.0, 143.0, 256.0, 105.0, 171.0, 436...
448      [109.0, 114.0, 128.0, 2.0, 143.0, 254.0, 256.0...
689      [128.0, 143.0, 258.0, 254.0, 256.0, 100.0, 508...
912      [508.0, 109.0, 114.0, 128.0, 143.0, 310.0, 39....
935      [39.0, 4.0, 145.0, 336.0, 53.0, 13.0, 44.0, 18...
1711     [309.0, 308.0, 259.0, 53.0, 231.0, 352.0, 437....
1893     [114.0, 109.0, 128.0, 143.0, 310.0, 256.0, 100...
1952     [508.0, 109.0, 114.0, 128.0, 2.0, 310.0, 39.0,...
1955     [114.0, 128.0, 2.0, 143.0, 508.0, 105.0, 310.0...
2025     [114.0, 258.0, 100.0, 256.0, 109.0, 508.0, 39..

In [34]:
def getKrecomFinal(row, k):
    """
        Funcion que saca las k mejores recomendaciones para el usuario
        Lo que hace es coger los primeros k valores de la lista de recomendaciones
    """
    if k == 1:
        value = list()
        value.append(row['recommendation'][:k])
        return value
    else:
        return row['recommendation'][:k]

In [35]:
# ahora saco los k mejores problemas para cada usuario
df_recommend['recommendation'] = df_recommend.apply(lambda row: getKrecomFinal(row, k), axis=1)

print(df_recommend)

                                            recommendation
user_id                                                   
18       [109.0, 128.0, 39.0, 143.0, 2.0, 258.0, 254.0,...
25       [346.0, 503.0, 114.0, 254.0, 100.0, 256.0, 171...
60       [508.0, 109.0, 114.0, 128.0, 143.0, 256.0, 258...
130      [508.0, 114.0, 128.0, 256.0, 254.0, 258.0, 255...
414      [309.0, 128.0, 346.0, 39.0, 503.0, 256.0, 100....
443      [128.0, 346.0, 143.0, 256.0, 105.0, 171.0, 436...
448      [109.0, 114.0, 128.0, 2.0, 143.0, 254.0, 256.0...
689      [128.0, 143.0, 258.0, 254.0, 256.0, 100.0, 508...
912      [508.0, 109.0, 114.0, 128.0, 143.0, 310.0, 39....
935      [39.0, 4.0, 145.0, 336.0, 53.0, 13.0, 44.0, 18...
1711     [309.0, 308.0, 259.0, 53.0, 231.0, 352.0, 437....
1893     [114.0, 109.0, 128.0, 143.0, 310.0, 256.0, 100...
1952     [508.0, 109.0, 114.0, 128.0, 2.0, 310.0, 39.0,...
1955     [114.0, 128.0, 2.0, 143.0, 508.0, 105.0, 310.0...
2025     [114.0, 258.0, 100.0, 256.0, 109.0, 508.0, 39..

In [36]:
# hago el filtro para los usuarios a los que tengo que recomendar
df_users_eval = df_users_eval[df_users_eval['user_id'].isin(user_list_to_recommend)]
print(df_users_eval)

                                       list_problem_id  user_id
4               [241, 128, 124, 256, 404, 469, 70, 86]       18
5                    [33, 29, 254, 308, 445, 253, 166]       25
8    [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...       60
13                       [254, 253, 308, 166, 49, 445]      130
22   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...      414
26             [134, 187, 256, 251, 90, 174, 147, 438]      443
27   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...      448
34                     [327, 23, 150, 349, 336, 6, 49]      689
44   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...      912
46          [95, 383, 262, 122, 44, 81, 252, 145, 336]      935
70   [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...     1711
73                       [471, 469, 355, 95, 243, 436]     1893
78         [374, 195, 109, 436, 95, 10, 446, 340, 379]     1952
80                            [44, 213, 136, 162, 243]     1955
90             [136, 253, 187, 342, 203,

In [37]:
# primero voy a ordenar la lista de usuarios a recomendar
user_list_to_recommend.sort()

list_eval_problems = df_users_eval['list_problem_id'].tolist()
list_recom_problems = df_recommend['recommendation'].tolist()


# meto toda la informacion en un dataframe para obtener las metricas
set_df_metric = {'user_id': user_list_to_recommend, 'eval_problems': list_eval_problems, 'recom_problems': list_recom_problems}
metric_df = pd.DataFrame.from_dict(set_df_metric)

print(metric_df)

                                        eval_problems  \
0              [241, 128, 124, 256, 404, 469, 70, 86]   
1                   [33, 29, 254, 308, 445, 253, 166]   
2   [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...   
3                       [254, 253, 308, 166, 49, 445]   
4   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...   
5             [134, 187, 256, 251, 90, 174, 147, 438]   
6   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...   
7                     [327, 23, 150, 349, 336, 6, 49]   
8   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...   
9          [95, 383, 262, 122, 44, 81, 252, 145, 336]   
10  [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...   
11                      [471, 469, 355, 95, 243, 436]   
12        [374, 195, 109, 436, 95, 10, 446, 340, 379]   
13                           [44, 213, 136, 162, 243]   
14            [136, 253, 187, 342, 203, 33, 441, 508]   
15  [136, 253, 187, 33, 441, 342, 209, 162, 155, 4...   
16                           [1

In [38]:
def one_hit(row):
    """
        Funcion que implementa la metrica one hit. Devuelve un 1 si para un usuarios dado, al menos uno
        de los problemas que se le ha recomendado ha sido realizado por ese usuario en el evaluation_set. 
        Cero si no hay ningun problema de los recomendados que haya sido realizado por el usuario
    """
    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    if len(num_problems_common) >= 1:
        return 1
    else:
        return 0

In [39]:
def mrr(row): 
    """
        Funcion que va a implementar la metrica de evaluacion mrr:
        mrr = 1/ranki, donde ranki es la posicion del primer item correcto
    """

    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    if len(num_problems_common) >= 1:

        # hago la busqueda del primer elemento que esta en la lista de recomendados
        fst_correct_item = -1
        encontrado = False
        i = 0
        while (i < len(row['recom_problems'])) and (encontrado == False):
            if row['recom_problems'][i] in row['eval_problems']:
                # fst_correct_item = row['recom_problems'][i]
                # print(fst_correct_item)
                ranki = i + 1
                encontrado = True
            else:
                i = i + 1
                
        return (1/ranki)

    else:
        return 0


In [40]:
def precision(row):
    """
        Funcion que va a implementar la metrica precision en k: 
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los recomendados
    """
    
    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row['recom_problems']))

In [41]:
def recall(row):
    """
        Funcion que implementa la metrica recall
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los evaluados
    """
    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row['eval_problems']))


def f1(row):
    """
        Funcion que calcula el f1 en funcion de precision y recall
    """
    denominador = row['precision'] + row['recall']
    
    if denominador == 0:
        return 0
    else:
        return (2 * row['precision'] * row['recall']) / denominador

In [42]:
# ahora voy a calcular una metrica para cada usuario
metric_df['one_hit'] = metric_df.apply(lambda row: one_hit(row), axis=1)
metric_df['mrr'] = metric_df.apply(lambda row: mrr(row), axis=1)
metric_df['precision'] = metric_df.apply(lambda row: precision(row), axis=1)
metric_df['recall'] = metric_df.apply(lambda row: recall(row), axis=1)
metric_df['f1'] = metric_df.apply(lambda row: f1(row), axis=1)
print(metric_df)

                                        eval_problems  \
0              [241, 128, 124, 256, 404, 469, 70, 86]   
1                   [33, 29, 254, 308, 445, 253, 166]   
2   [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...   
3                       [254, 253, 308, 166, 49, 445]   
4   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...   
5             [134, 187, 256, 251, 90, 174, 147, 438]   
6   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...   
7                     [327, 23, 150, 349, 336, 6, 49]   
8   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...   
9          [95, 383, 262, 122, 44, 81, 252, 145, 336]   
10  [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...   
11                      [471, 469, 355, 95, 243, 436]   
12        [374, 195, 109, 436, 95, 10, 446, 340, 379]   
13                           [44, 213, 136, 162, 243]   
14            [136, 253, 187, 342, 203, 33, 441, 508]   
15  [136, 253, 187, 33, 441, 342, 209, 162, 155, 4...   
16                           [1

In [43]:
# Para crear un archivo grafo para GEPHI


# creo los enlaces a partir de la informacion de los nodos
# links = createLinks(grouped, nodes)


# aqui creo el grafo 
# graph = create_graph_nx(nodes, links)
# nx.write_gexf(graph,"grafo.gexf")

In [44]:
# calculo la media de las metricas

result_one_hit = metric_df['one_hit'].mean()
result_precision = metric_df['precision'].mean()
result_mrr = metric_df['mrr'].mean()
result_recall = metric_df['recall'].mean()
result_f1 = metric_df['f1'].mean()

print("One hit ----------")
print(result_one_hit)
print("Precision ----------")
print(result_precision)
print("Mrr  ----------")
print(result_mrr)
print("Recall  ----------")
print(result_recall)
print("F1  ----------")
print(result_f1)

One hit ----------
0.6615384615384615
Precision ----------
0.11230769230769228
Mrr  ----------
0.27020146520146515
Recall  ----------
0.10544879598473549
F1  ----------
0.10204153686183615


In [45]:

f = open("C:/hlocal/TFM/nodos_problemas", 'a')
f.write(str(result_one_hit) + '\t' + str(result_precision) + '\t' + str(result_mrr) + '\t' + str(result_recall) + '\t' +  str(result_f1) + '\n') 
f.close()